# In this notebook, we will search some twitte message

# Pre-requise: Create twitter developer account and generate api credentials
Step 1: Apply for a Twitter Developer Account

Go to the Twitter developer site to apply for a developer account. Here, you have to select the Twitter user responsible for this account. It should probably be you or your organization. Here’s what this page looks like:



In [13]:
import tweepy as tw
import json

# Download tweets
## Step1. Setup the twitter api credential


In [2]:
consumer_key = "9F2cDP6mBO001MJtFyLybWGqT"
consumer_secret = "czoiPDtFaNrUboAQEPPXpb9EMwbTgW7WB6bZYxpBljy2Ozqrqc"
access_token = "207529473-0NYW8AbMELgWLUOtB8zCLYKSH2T3BPzb4iqr15Df"
access_token_secret = "TsIFoTd0yILe2pIjDczFf56ocDzxoQfaGxnDS10asw8qz"

## Step2. Create an instance of the twitter client

In [3]:
client_auth = tw.OAuthHandler(consumer_key, consumer_secret)
client_auth.set_access_token(access_token, access_token_secret)
api = tw.API(client_auth, wait_on_rate_limit=True, retry_count=5, retry_delay=1)

In [4]:
try:
    api.verify_credentials()
    print("Authentication OK")
except Exception as e:
    print("Error during authentication")

Authentication OK


## Step3. Get tweets that you are intrested in 

We have created the twitter client, now we are ready to search the tweets messages. Here we use the search_tweets to find all the tweets that contains certain key words of certain language. We don't use date to filter tweets because the search index has a 7-day limit. In other words, no tweets will be found for a date older than one week.
For more details about the search_tweets
https://docs.tweepy.org/en/stable/api.html?highlight=search%20tweet#tweepy.API.search_tweets

If you want to search older tweets, you can use the search_full_archive method.
https://docs.tweepy.org/en/stable/api.html?highlight=search%20tweet#tweepy.API.search_full_archive

In [28]:
# filter the search result by using key words and date
search_words = "#insee"
# date_since = "2021-11-24
language="fr"

In [36]:
tweets = api.search_tweets(q=search_words,lang="fr",result_type="mixed", count=1000)
# print(tweets[0]._json)
for tweet in tweets:
    tweet_dict=tweet._json
    text=tweet_dict.get("text")
    user_name=tweet_dict.get("user").get("name")
    date=tweet_dict.get("created_at")
    print(f"name:{user_name}, message: {text}")

{'created_at': 'Wed Nov 24 13:38:12 +0000 2021', 'id': 1463502210285940736, 'id_str': '1463502210285940736', 'text': 'https://t.co/2nt1wV9P6U\nRFR - L’inflation arrive en France #france #croissance #inflation #insee #conjoncture… https://t.co/s47ETAqWBO', 'truncated': True, 'entities': {'hashtags': [{'text': 'france', 'indices': [59, 66]}, {'text': 'croissance', 'indices': [67, 78]}, {'text': 'inflation', 'indices': [79, 89]}, {'text': 'insee', 'indices': [90, 96]}, {'text': 'conjoncture', 'indices': [97, 109]}], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/2nt1wV9P6U', 'expanded_url': 'https://richesflores.com/?p=15494', 'display_url': 'richesflores.com/?p=15494', 'indices': [0, 23]}, {'url': 'https://t.co/s47ETAqWBO', 'expanded_url': 'https://twitter.com/i/web/status/1463502210285940736', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [111, 134]}]}, 'metadata': {'iso_language_code': 'fr', 'result_type': 'recent'}, 'source': '<a href="https://richesflor